In [2]:
!pip install ipynb
import ipynb
import numpy as np

In [3]:
from ipynb.fs.defs.speck import make_train_data
from ipynb.fs.defs.speck import real_differences_data

In [4]:
import keras
from keras.models import Model
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers import Dense,Conv1D,Input,Reshape,Add,Flatten,Concatenate,Permute,BatchNormalization,Activation

In [5]:
def cyclic_lr(num_epochs, high_lr, low_lr):
  res = lambda i: low_lr + ((num_epochs-1) - i % num_epochs)/(num_epochs-1) * (high_lr - low_lr);
  return(res)

lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001))

In [6]:
def make_resnet(num_blocks=2, num_filters=32, num_outputs=1, d1=64, d2=64, word_size=16, ks=3,depth=10, reg_param=0.0001, final_activation='sigmoid'):
  #Input and preprocessing layers
  inp = Input(shape=(num_blocks * word_size * 2,));
  rs = Reshape((2 * num_blocks, word_size))(inp);
  perm = Permute((2,1))(rs);
  #add a single residual layer that will expand the data to num_filters channels
  #this is a bit-sliced layer
  conv0 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(perm);
  conv0 = BatchNormalization()(conv0);
  conv0 = Activation('relu')(conv0);
  #add residual blocks
  shortcut = conv0;
  for i in range(depth):
    conv1 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut);
    conv1 = BatchNormalization()(conv1);
    conv1 = Activation('relu')(conv1);
    conv2 = Conv1D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1);
    conv2 = BatchNormalization()(conv2);
    conv2 = Activation('relu')(conv2);
    shortcut = Add()([shortcut, conv2]);
  #add prediction head
  flat1 = Flatten()(shortcut);
  dense1 = Dense(d1,kernel_regularizer=l2(reg_param))(flat1);
  dense1 = BatchNormalization()(dense1);
  dense1 = Activation('relu')(dense1);
  dense2 = Dense(d2, kernel_regularizer=l2(reg_param))(dense1);
  dense2 = BatchNormalization()(dense2);
  dense2 = Activation('relu')(dense2);
  out = Dense(num_outputs, activation=final_activation, kernel_regularizer=l2(reg_param))(dense2);
  model = Model(inputs=inp, outputs=out);
  return(model);

In [11]:
res=make_resnet()

res.compile(optimizer='adam',loss='mse',metrics=['acc'])
X,Y=make_train_data(1000000,nr=5)

res.fit(X,Y,epochs=20,batch_size=5000,callbacks=lr)

In [8]:
x_test,y_test=real_differences_data(100000,nr=5)

In [12]:
z=res.predict(x_test,batch_size=10000).flatten()
zbin=(z>0.5)
n=len(z)
acc=np.sum(zbin==y_test)/n
print(acc)

In [13]:
res.evaluate(x_test,y_test,batch_size=10000)